In [10]:
from kamiwaza_client import KamiwazaClient
import openai
from pydantic import BaseModel

In [11]:
client = KamiwazaClient("http://34.230.49.204:7777/api/")
client.serving.list_active_deployments()

[ActiveModelDeployment(id=UUID('9974c1d7-444d-4f78-9bf6-30ecef701d43'), m_id=UUID('c45afd07-17b5-4509-b026-33c43e3b5467'), m_name='Qwen3-32B-AWQ', status='DEPLOYED', instances=[ModelInstance:
 ID: 70b4c6ea-41fa-458a-8e0a-2555c262a654
 Deployment ID: 9974c1d7-444d-4f78-9bf6-30ecef701d43
 Status: DEPLOYED
 Listen Port: 32775], lb_port=51105, endpoint='http://34.230.49.204:51105/v1')]

In [12]:
openai_client = client.openai.get_client('Qwen3-32B-AWQ')

In [13]:
# Create a streaming chat completion
response = openai_client.chat.completions.create(
    messages=[
        {"role": "user", "content": "A coin is chosen at random from a bag containing one fair coin, one coin with heads on both sides, and one coin with tails on both sides. You flip the chosen coin once and it lands heads. What is the probability you picked the coin with two heads?"}
    ],
    model="model",
    stream=True 
)

# display the stream
for chunk in response:
    delta = chunk.choices[0].delta

    # Thought tokens (cyan)
    if getattr(delta, "reasoning_content", None):
        print("\033[36m" + delta.reasoning_content + "\033[0m", end="", flush=True)

    # Final-answer tokens (default colour)
    if delta.content:
        print(delta.content, end="", flush=True)



2025-06-11 12:13:59,484 - httpx - INFO - HTTP Request: POST http://34.230.49.204:51105/v1/chat/completions "HTTP/1.1 200 OK"



Okay, so I have this probability problem here. Let me try to figure it out step by step. The problem says:

"A coin is chosen at random from a bag containing one fair coin, one coin with heads on both sides, and one coin with tails on both sides. You flip the chosen coin once and it lands heads. What is the probability you picked the coin with two heads?"

Alright, so first, let me make sure I understand the problem correctly. There are three coins in total:

1. A fair coin: which has one head (H) and one tail (T).
2. A two-headed coin: both sides are heads (H, H).
3. A two-tailed coin: both sides are tails (T, T).

We randomly pick one of these three coins, flip it once, and observe that it lands heads. We need to find the probability that the selected coin is the two-headed one, given this outcome.

This seems like a classic case for Bayes' Theorem. Let me recall Bayes' Theorem: 

P(A|B) = [P(B|A) * P(A)] / P(B)

Where:
- P(A|B) is the probability of event A given B,
- P(B|A) is the